### Import water supply data & create supply table
Here we download the raw supply data for years 2000, 2005, and 2010 from from downscaled CMIP5 hydrology projections ([link](http://gdo-dcp.ucllnl.org/downscaled_cmip_projections/techmemo/BCSD5HydrologyMemo.pdf)).

These data include monthly estimates of runoff, precipitation, evapotranspiration, and soil moisture content at a 1/8th degree spatial resolution across the US for the period of 1950 to 2099. Estimates are provided for 21 different climate projection ensembles applied to the Variable Infiltration Capacity (VIC) Macroscale Hydrologic Model ([link](http://vic.readthedocs.io/en/master/)); see the PDF document for a complete list. For demonstration purposes, this project uses the National Center for Atmospheric Research CCSM4 2.6 projection ensembles as the base data for water supply figures. 
 
The steps involved include:

* Download monthly runoff (total_runoff), precipitation (pr), evapotranspiration (et), and soil moisture content (smc) data, in NetCDF format, from a central data repository ([link](ftp://gdo-dcp.ucllnl.org/pub/dcp/archive/cmip5/hydro/BCSD_mon_VIC_nc/ccsm4_rcp26_r1i1p1/)) for a given sample year (2000, 2005, and 2010).

* For each year and parameter combination:

    * Extract the monthly data from the downloaded NetCDF files into 4-dimensional NumPy arrays (time, parameter value, latitude, longitude).

    * Collapse the time dimension (months) into annual sums, resulting in a 3-dimensional array for each parameter, i.e. a single annual value for each 1/8th degree coordinate pair: rows = latitudes, columns = longitudes.

    * Re-lable columns as longitude values and insert a column of latitude values. Then melt the table into a listing of lat, long, and value. 

    * Combines these 3-dimensional arrays, one for each parameter, into a single data frame listing parameter value, latitude, and longitude. 

    * Spatially join state FIPS codes to the data frame, using a county shapefile stored in the data folder. 

* Summarize supply values on FIPS to create a table that can be joined to other county level data:

| YEAR | FIPS | Precip | ET | Runoff | SoilMoisture | TotalSupply | 
| :---: | :---: | :---: | :---: | :---: | :---: | :---: |
| 2000 | 01001 | 0 | 0 | 0 | 0 | 0 |

In [3]:
#Import libraries
import sys, os, glob, time, datetime, urllib
import numpy as np
import pandas as pd
import netCDF4

In [85]:
#Function for pulling in netCDF4 format and converting to a dataframe
def getSupplyData(year, dfFIPS):
    
    #Get URLs for NCAR 2.6 scenario ensembles: runoff(ro), precipitation(pr), evapotranspiration(et), soil moisture (sm)
    baseURL = 'ftp://gdo-dcp.ucllnl.org/pub/dcp/archive/cmip5/hydro/BCSD_mon_VIC_nc/ccsm4_rcp26_r1i1p1/'
    baseURL2 = 'ftp://gdo-dcp.ucllnl.org/pub/dcp/archive/cmip5/hydro/BCSD_mon_forc_nc/ccsm4_rcp26_r1i1p1/' #(for precip)
    roURL = baseURL + "conus_c5.ccsm4_rcp26_r1i1p1.monthly.total_runoff.{}.nc".format(year)
    prURL = baseURL2 + "conus_c5.ccsm4_rcp26_r1i1p1.monthly.pr.{}.nc".format(year)
    etURL = baseURL + "conus_c5.ccsm4_rcp26_r1i1p1.monthly.et.{}.nc".format(year)
    smURL = baseURL + "conus_c5.ccsm4_rcp26_r1i1p1.monthly.smc.{}.nc".format(year)
    
    #Save as a dictionary (for clearer scripting)
    paramDict = {'runoff': roURL, 'precip': prURL, 'et':etURL, 'soil':smURL}

    #These lines fix an issue with slow network connections
    import socket
    socket.setdefaulttimeout(30)

    #Loop through each file and create an annual sum array; add it to a dictionary
    for param, url in paramDict.items():
        print "->Downloading {} data for year {}".format(param, year),
        
        #Retrieve the data file from the ftp server
        urllib.urlretrieve(url,"tmpData.nc")
        
        #Convert to netCDF object
        nc = netCDF4.Dataset("tmpData.nc",mode='r')
        
        #Get the parameter name and its values
        param_name = nc.variables.keys()[-1]
        param_vals = nc.variables.values()[-1]
        
        #Collapse the monthly values into a single 3d data frame
        dfParam = pd.DataFrame(param_vals[:,:,:].sum(axis=0))
        
        #Create latitude and longitude arrays (for the first element only)
        if url == roURL:
            dfLats = pd.DataFrame(nc.variables['latitude'][:])
            dfLons = pd.DataFrame(nc.variables['longitude'][:])
            
        #Close the nc object
        nc.close()
        
        #Delete the nc file
        os.remove("tmpData.nc")
        
        #Update
        urllib.urlcleanup()
        print "  ...**complete!**"

        #Melt the data into a 3 column, 2d data frame
        '''At this point, the dfParam data frame contains columns for each 1/8d longitude
           and rows for each 1/8d of latitude. The 'melt' procedure below collapses this into
           a three column table of lat,lon,value for the current parameter (e.g. runoff)
        '''
        dfParam.columns = dfLons[0].values.tolist() #Set column names to longitudes
        dfParam['LAT'] = dfLats[0].values.tolist()  #Add column of longitudes
        df = pd.melt(dfParam,id_vars=['LAT'],var_name='LON',value_name=param_name)

        #Append to dataframe, if not the first element
        '''Each attribute will become its own column in the final dataframe. So after assembling
           the first dataframe (runoff), we can just append the others to it. We also join the FIPS
           codes (created in a previous script) to the runoff dataframe so we can summarize by states
           or counties later. 
        '''
        if param == 'runoff': #If its the first in the series of runoff, precip, et, soil moisture...
            #Copy to the year dataframe
            dfYear = df.copy(deep=True)
            #Append fips values from FIPS dataframe
            dfYear['FIPS'] = dfFIPS.COFIPS
            dfYear['STFIPS'] = dfFIPS.STATEFIPS
            #Ensure that FIPS retain 5 digits (2 digits for state FIPS)
            dfYear['FIPS'] = dfYear['FIPS'].apply(lambda x: str(x).zfill(5))
            dfYear['STFIPS'] = dfYear['STFIPS'].apply(lambda x: str(x).zfill(2))
        else:
            #Add column to the year data frame
            dfYear[param_name] = df[param_name]
            
    #Add year to the master data frame
    dfYear.insert(1,'YEAR',year)
    
    #Return the dataframe
    return dfYear

In [77]:
#Create a dataframe of FIPS codes
fipsFN = '../../Data/FIPS.csv'
dfFIPS = pd.read_csv(fipsFN,dtype=np.str,na_values=-1)

In [86]:
#Retrieve data frames for each sample year, using the function above
df2000 = getSupplyData(2000, dfFIPS)
df2005 = getSupplyData(2005, dfFIPS)
df2010 = getSupplyData(2010, dfFIPS)

->Downloading runoff data for year 2000   ...**complete!**
->Downloading et data for year 2000   ...**complete!**
->Downloading precip data for year 2000   ...**complete!**
->Downloading soil data for year 2000   ...**complete!**
->Downloading runoff data for year 2005   ...**complete!**
->Downloading et data for year 2005   ...**complete!**
->Downloading precip data for year 2005   ...**complete!**
->Downloading soil data for year 2005   ...**complete!**
->Downloading runoff data for year 2010   ...**complete!**
->Downloading et data for year 2010   ...**complete!**
->Downloading precip data for year 2010   ...**complete!**
->Downloading soil data for year 2010   ...**complete!**


In [90]:
dfX = df2000.dropna(how='any')
dfX.to_csv("df2000.csv",index_label='OID')

In [75]:
#Concatenate the tables
dfAllYears = pd.concat([df2000,df2005,df2010],ignore_index=True)
dfAllYears.shape[0]

307692

In [76]:
dfAllYears.FIPS.unique()

array(['-0001', '53009', '41015', ..., '23009', '23029', '00nan'], dtype=object)

In [73]:
#Drop rows with invalid FIPS
dfAllYears = dfAllYears[dfAllYears.FIPS == '-0001']
dfAllYears.shape[0]

5355

In [74]:
#Drop rows with null data
dfAllYears.dropna(how='any',inplace=True)
dfAllYears.shape[0]

2673

In [65]:
dfAllYears.head()

,LAT,YEAR,LON,total_runoff,FIPS,et,pr,smc
182,47.9375,2000,-124.688,2216.291016,41011,741.627014,2860.095459,13090.798828
183,48.0625,2000,-124.688,2181.373291,41011,738.717041,2823.416016,13044.400391
184,48.1875,2000,-124.688,2167.416016,41011,745.596985,2814.455078,12944.500000
185,48.3125,2000,-124.688,2310.494873,41019,674.234985,2894.345947,12231.600586
186,48.4375,2000,-124.688,2341.796143,41019,694.606018,2951.890137,12462.999023


In [66]:
#Compute mean values for each county
groupCounty = dfAllYears.groupby(('YEAR','FIPS')).mean()

In [67]:
groupCounty

LAT  total_runoff          et           pr          smc
YEAR FIPS                                                                
2000 -0001  41.262416    320.287476  490.845947   755.800110  3985.989990
     00nan  38.519160    331.999573  721.956177  1061.081299  2977.788086
     01001  40.437500     17.965500  438.774841   467.279633  1702.066650
     01003  38.827500     35.501320  390.926849   419.062531  2430.687988
     01005  38.062500     38.289249  479.381592   495.510925  3127.558350
     01007  37.796875     36.464249  388.448120   413.576508  3758.399902
     01009  40.968750     12.300250  445.373016   451.275391  2115.800049
     01011  37.416667     26.026667  445.433838   461.893494  2645.566650
     01013  39.687500     23.211126  442.455383   449.265045  1670.949951
     01015  36.590278     32.224445  422.308563   437.650452  2639.944580
     01017  39.900000     48.533001  479.802795   504.117065  3665.050049
     01019  37.756944     36.597889  467.771545   507.485138  2901.455566
     01021  38.656250     21.636501  430.691010   440.643707  2187.787598
     01023  41.109375     34.478500  398.890625   427.494781  2941.100098
     01025  38.750000     27.051500  416.472778   427.852692  2269.205566
     01027  34.758929     29.561285  404.409576   415.462982  3265.728516
     01029  36.125000     42.566750  419.057983   462.592865  3538.087402
     01031  39.937500     32.312626  417.887238   445.320404  2530.987549
     01033  38.612500     38.957802  367.932800   419.359344  3377.699951
     01035  37.093750     39.403084  405.517517   439.247345  3103.408203
     01037  36.812500     45.495003  373.766846   449.087677  3778.400146
     01039  36.062500     29.795626  398.240021   412.267426  3084.662598
     01041  41.026786     46.420288  405.969574   409.986023  2275.842773
     01043  40.562500     38.242554  427.174347   444.245728  2999.388916
     01045  35.812500     36.769001  459.494995   466.604248  2775.750000
     01047  38.945312     48.410561  405.563995   453.105865  2800.543701
     01049  38.050000     34.954899  465.528198   492.122864  2594.520020
     01051  36.537500     33.403599  391.608002   456.144897  2485.580078
     01053  39.500000     33.125786  412.596649   421.469513  2518.514160
     01055  38.737500     31.621199  409.305420   451.398071  3051.120117
...               ...           ...         ...          ...          ...
2010 55129  37.166667     37.645668  443.963593   460.061615  2238.316650
     55131  42.125000     36.875000  478.002991   491.368103  3176.649902
     55133  38.104167     53.601665  486.437012   527.589355  4016.688721
     55135  39.593750     39.544666  527.232178   537.806702  2964.066650
     55137  40.700000     37.658501  516.264404   515.195801  2914.840088
     55139  38.419643     49.707142  589.721985   585.174622  3328.699951
     55141  38.687500     22.979876  548.284912   508.083069  2157.800049
     56001  43.046875    329.193359  380.082092   781.120911  3845.764893
     56003  39.841570    137.892578  397.053802   582.585327  4202.630371
     56005  37.426862     40.719936  273.812378   355.232971  2530.166016
     56007  41.228639    112.988380  371.500946   520.794983  3341.492432
     56009  38.548214     18.416086  207.705170   241.417145  1753.402832
     56011  40.062500     46.169716  371.151093   466.854156  2916.687744
     56013  40.037946    169.469742  405.169434   625.045349  3623.619629
     56015  39.666667     86.153923  440.572327   580.557983  3301.283203
     56017  40.547500    187.923401  440.400757   671.552490  5096.956055
     56019  39.842500     56.374962  324.837433   415.652740  3173.838135
     56021  40.897321    149.442795  338.629120   546.099121  2965.442871
     56023  41.383929     73.060547  340.208069   447.387695  3400.950928
     56025  41.383929    247.382919  378.627106   687.033997  3983.125488
     56027  40.348790     97.577614  358.375427   504.534241  3352.445068
     56029  42.585489 

In [12]:
df2000.dropna(thresh=2)
df2000.iloc[2100:2110]

,LAT,YEAR,LON,total_runoff,FIPS,pr,et,smc
2100,37.9375,2000,-123.562,NaN,53077,NaN,NaN,NaN
2101,38.0625,2000,-123.562,NaN,53077,NaN,NaN,NaN
2102,38.1875,2000,-123.562,NaN,53077,NaN,NaN,NaN
2103,38.3125,2000,-123.562,NaN,53077,NaN,NaN,NaN
2104,38.4375,2000,-123.562,NaN,53077,NaN,NaN,NaN
2105,38.5625,2000,-123.562,NaN,53077,NaN,NaN,NaN
2106,38.6875,2000,-123.562,413.488983,53077,1125.675903,826.181091,3425.299805
2107,38.8125,2000,-123.562,458.260986,53077,1164.038696,810.178040,3536.900146
2108,38.9375,2000,-123.562,548.063049,53033,1211.572876,755.596985,4223.500000
2109,39.0625,2000,-123.562,511.115967,53037,1178.547119,770.424011,4129.299805


In [ ]:
#Grab the FIPS data and create a dataframe from it
print "Getting record FIPS data"
fipsURL = "https://raw.githubusercontent.com/johnpfay/USWaterAccounting/Data/FIPS.csv"
dfFIPS = pd.read_csv(fipsURL,dtype=np.str)

In [ ]:
fipsURL = 'https://www2.census.gov/geo/docs/reference/codes/files/national_county.txt'
dfFIPS = pd.read_csv(fipsURL,dtype=np.str)

In [ ]:
#Initialize the output file and write the header line
print "Initializing the output file"
outFile = open("..\..\Scratch\HydroData.csv",'wt')
outFile.write("YEAR,LONGITUDE,LATITUDE,COFIPS,STFIPS,RUNOFF,PRECIP,ET,SME\n")

In [ ]:
#Get precip data
year = 2000
baseURL2 = 'ftp://gdo-dcp.ucllnl.org/pub/dcp/archive/cmip5/hydro/BCSD_mon_forc_nc/ccsm4_rcp26_r1i1p1/'
prURL = baseURL2 + "conus_c5.ccsm4_rcp26_r1i1p1.monthly.pr.{}.nc".format(year)
urllib.urlretrieve(prURL,"tmpData.nc")

In [ ]:
#Create a netCDF object
nc = netCDF4.Dataset("tmpData.nc",mode='r')

In [ ]:
#Get the parameter name and its values (the last dimension in the nc object)
param_name = nc.variables.keys()[-1]
param_vals = nc.variables.values()[-1]

In [ ]:
#Create dataframes of latitude and longitude values
dfLats = pd.DataFrame(nc.variables['latitude'][:])
dfLons = pd.DataFrame(nc.variables['longitude'][:])

In [ ]:
#Create a dataframe of the parameter values, summed for all month records
# The columns here are longitudes and rows are latitudes
dfParam = pd.DataFrame(param_vals[:,:,:].sum(axis=0))
dfParam.columns = dfLons[0].values.tolist()
dfParam['LAT'] = dfLats[0].values.tolist()
df = pd.melt(dfParam,id_vars=['LAT'],var_name='LON',value_name=param_name)
df.dropna(inplace=True)
df.to_csv("foo.csv",index_name="OID")

In [ ]:
#Set the year to process
year = 2000
print "Processing year {}".format(year)

#Get urls for NCAR 2.6 scenario ensembles: runoff(ro), precipitation(pr), evapotranspiration(et), soil moisture (sm)
baseURL = 'ftp://gdo-dcp.ucllnl.org/pub/dcp/archive/cmip5/hydro/BCSD_mon_VIC_nc/ccsm4_rcp26_r1i1p1/'
baseURL2 = 'ftp://gdo-dcp.ucllnl.org/pub/dcp/archive/cmip5/hydro/BCSD_mon_forc_nc/ccsm4_rcp26_r1i1p1/'
roURL = baseURL + "conus_c5.ccsm4_rcp26_r1i1p1.monthly.total_runoff.{}.nc".format(year)
prURL = baseURL2 + "conus_c5.ccsm4_rcp26_r1i1p1.monthly.pr.{}.nc".format(year)
etURL = baseURL + "conus_c5.ccsm4_rcp26_r1i1p1.monthly.et.{}.nc".format(year)
smURL = baseURL + "conus_c5.ccsm4_rcp26_r1i1p1.monthly.smc.{}.nc".format(year)

#These lines fix an issue with slow network connections
import socket
socket.setdefaulttimeout(30)

#Loop through each file and create an annual sum array; add it to a dictionary
dataDict = {}
url = roURL
print "...downloading data from " + url
#Retrieve the data file from the ftp server
urllib.urlretrieve(url,"tmpData.nc")

#Convert to netCDF object
nc = netCDF4.Dataset("tmpData.nc",mode='r')

#Add the lats and lons array to the dictionary
dataDict["lats"] = nc.variables['latitude'][:]
dataDict["lons"] = nc.variables['longitude'][:]

#Get the parameter name and its values
param_name = nc.variables.keys()[-1]
param_vals = nc.variables.values()[-1]

#Collapse the monthly values into a single array
dataDict[param_name] = param_vals[:,:,:].sum(axis=0)

#Close the nc object
nc.close()

#Delete the nc file
os.remove("tmpData.nc")

#Update
urllib.urlcleanup()
print "....complete"

#Write array values as X,Y table
#Create lons and lats array
lons = dataDict["lons"]
lats = dataDict["lats"]

#Initialize the index to retrieve FIPS codes
idxFIPS = 0

In [ ]:
import pip
pip.main(['install','geopy'])